In [2]:
# Import Dependencies
import pandas as pd
import numpy as np
import re
import spacy
from spacy.lang.en import English
from spacy.attrs import ORTH, LEMMA, NORM, TAG
from clean import replace_emoticons, clean

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import ComplementNB, MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.svm import SVC

from joblib import dump

In [3]:
# Import spacy nlp library
nlp = spacy.load('en_core_web_sm',parser=False, entity=False)

## Import Dataset

In [4]:
twitter_full_df = pd.read_csv("Resources/sent_analysis_dataset.csv", error_bad_lines=False)

b'Skipping line 8836: expected 4 fields, saw 5\n'
b'Skipping line 535882: expected 4 fields, saw 7\n'


In [5]:
# Select slice to test
twitter_df = twitter_full_df.loc[:100000]

In [7]:
twitter_df = twitter_full_df.sample(frac=.25)

## Clean Tweets

In [6]:
# Drop and rename columns
twitter_df = twitter_df.drop(["ItemID", "SentimentSource"], axis = 1)
twitter_df = twitter_df.rename(columns = {"SentimentText": "Text"})

In [7]:
# Check columns for missing data
twitter_df.isnull().sum()

Sentiment    0
Text         0
dtype: int64

In [7]:
# Verify data are of correct type
twitter_df.dtypes

Sentiment     int64
Text         object
dtype: object

In [6]:
# Verify sentiment column has appropriate data
twitter_df["Sentiment"].unique()

array([0, 1])

In [8]:
# Clean text of tweets using previously defined clean_tweet function
twitter_df["Text"] = twitter_df["Text"].map(lambda x: clean(x))

In [9]:
twitter_df.head()

,Sentiment,Text
0,0,is so sad for my apl friend
1,0,missed the new moon trailer
2,1,omg its already
3,0,omgaga im sooo im gunna cry ve been at this de...
4,0,think mi bf is cheating on me


In [13]:
# Remove empty text strings
twitter_df = twitter_df.dropna(axis=0, subset=["Text"])

In [34]:
# Backup cleaned csv
twitter_df.to_csv("Resources/clean_dataset_25perc.csv")

In [13]:
#twitter_df = pd.read_csv("Resources/clean_dataset.csv")

In [14]:
#twitter_df = twitter_df[:250000]

## Split into testing and training datasets

In [15]:
# Split into testing and training datasets
X_train, X_test, y_train, y_test = train_test_split(twitter_df["Text"], twitter_df["Sentiment"])

In [ ]:
# Load English tokenizer, tagger, parser, NER and word vectors
parser = English()

# Creating our tokenizer function
def spacy_tokenizer(tweet):

    tokens = parser(tweet)

    # Lemmatizing each token and converting each token into lowercase
    tokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]

    # Removing stop words
    mytokens = [ word for word in mytokens if word not in stop_words ]

    # return preprocessed list of tokens
    return mytokens

In [ ]:
# Custom transformer using spaCy
class predictors(TransformerMixin):
    def transform(self, X, **transform_params):
        # Cleaning Text
        return [clean_text(text) for text in X]

    def fit(self, X, y=None, **fit_params):
        return self

    def get_params(self, deep=True):
        return {}

# Basic function to clean the text
def clean_text(text):
    # Removing spaces and converting text into lowercase
    return text.strip().lower()

In [ ]:
# Create vectorizer
tfidf_vector = TfidfVectorizer(tokenizer = spacy_tokenizer)

In [ ]:
# Logistic Regression Classifier
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()

# Create pipeline using Bag of Words
pipe = Pipeline([("cleaner", predictors()),
                 ('vectorizer', bow_vector),
                 ('classifier', classifier)])

# model generation
pipe.fit(X_train,y_train)

In [ ]:
from sklearn import metrics
# Predicting with a test dataset
predicted = pipe.predict(X_test)

# Model Accuracy
print("Logistic Regression Accuracy:",metrics.accuracy_score(y_test, predicted))
print("Logistic Regression Precision:",metrics.precision_score(y_test, predicted))
print("Logistic Regression Recall:",metrics.recall_score(y_test, predicted))

## Test different models

In [ ]:
### Multinomial Naive Bayes Classifier

In [ ]:
# Create transformation pipeline
pipeline = Pipeline([
    ("vectorize", TfidfVectorizer()),
    ("classify", MultinomialNB()),
])

# Set parameters for Gridsearch
parameters = {"vectorize__use_idf": (True, False),
              "vectorize__ngram_range": [(1,1), (1,2)],
              "vectorize__max_df" : (1, .8),
              "vectorize__norm": ("l1", "l2"),
              "classify__alpha": (.8, 1)
             }
mnb = GridSearchCV(pipeline, param_grid = parameters, n_jobs = -1, cv = 5, verbose = 1)

In [ ]:
mnb.fit(X_train,y_train)

In [ ]:
# Get best results
print(f"Best Score: {mnb.best_score_} with parameters: {mnb.best_params_}")
optimized_mnb = mnb.best_estimator_

In [ ]:
# Evaluate model using test data
predictions = optimized_mnb.predict(X_test)
print(f"Accuracy score: {accuracy_score(y_test, predictions)}")
print(f"Confusion matrix: {confusion_matrix(y_test, predictions)}")
print(f"Classification report: \n{classification_report(y_test, predictions)}")

### Complement Naive Bayes Classifier

In [6]:
# Create transformation pipeline
pipeline = Pipeline([
    ("vectorize", TfidfVectorizer()),
    ("classify", ComplementNB()),
])

# Set parameters for Gridsearch
parameters = {"vectorize__use_idf": (True, False),
              "vectorize__ngram_range": [(1,1), (1,2)],
              "vectorize__max_df" : (1, .8),
              "vectorize__norm": ("l1", "l2"),
              "classify__alpha": (.8, 1)
             }
cnb = GridSearchCV(pipeline, param_grid = parameters, n_jobs = -1, cv = 5, verbose = 1)

In [22]:
cnb.fit(X_train,y_train)

Fitting 5 folds for each of 32 candidates, totalling 160 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  8.0min
[Parallel(n_jobs=-1)]: Done 160 out of 160 | elapsed: 32.5min finished


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('vectorize', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=T...abulary=None)), ('classify', ComplementNB(alpha=1.0, class_prior=None, fit_prior=True, norm=False))]),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'vectorize__use_idf': (True, False), 'vectorize__ngram_range': [(1, 1), (1, 2)], 'vectorize__max_df': (1, 0.8), 'vectorize__norm': ('l1', 'l2'), 'classify__alpha': (0.8, 1)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

In [23]:
# Get best results
print(f"Best Score: {nb.best_score_} with parameters: {nb.best_params_}")
optimized_cnb = cnb.best_estimator_

Best Score: 0.7812020747395487 with parameters: {'classify__alpha': 1, 'vectorize__max_df': 0.8, 'vectorize__ngram_range': (1, 2), 'vectorize__norm': 'l2', 'vectorize__use_idf': False}


In [24]:
# Evaluate model using test data
predictions = optimized_cnb.predict(X_test)
print(f"Accuracy score: {accuracy_score(y_test, predictions)}")
print(f"Confusion matrix: {confusion_matrix(y_test, predictions)}")
print(f"Classification report: \n{classification_report(y_test, predictions)}")

Accuracy score: 0.7830241935483871
Confusion matrix: [[ 90726  25383]
 [ 28427 103464]]
Classification report: 
              precision    recall  f1-score   support

           0       0.76      0.78      0.77    116109
           1       0.80      0.78      0.79    131891

   micro avg       0.78      0.78      0.78    248000
   macro avg       0.78      0.78      0.78    248000
weighted avg       0.78      0.78      0.78    248000



### Multinomial Naive Bayes Classifier

In [28]:
# Create transformation pipeline
pipeline = Pipeline([
    ("vectorize", TfidfVectorizer()),
    ("classify", MultinomialNB()),
])

# Set parameters for Gridsearch
parameters = {"vectorize__use_idf": (True, False),
              "vectorize__ngram_range": [(1,1), (1,2)],
              "vectorize__max_df" : (1, .8),
              "vectorize__norm": ("l1", "l2"),
              "classify__alpha": (.8, 1)
             }
mnb = GridSearchCV(pipeline, param_grid = parameters, n_jobs = -1, cv = 5, verbose = 1)

In [29]:
mnb.fit(X_train,y_train)

Fitting 5 folds for each of 32 candidates, totalling 160 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  8.1min
[Parallel(n_jobs=-1)]: Done 160 out of 160 | elapsed: 33.7min finished


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('vectorize', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=T...        vocabulary=None)), ('classify', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))]),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'vectorize__use_idf': (True, False), 'vectorize__ngram_range': [(1, 1), (1, 2)], 'vectorize__max_df': (1, 0.8), 'vectorize__norm': ('l1', 'l2'), 'classify__alpha': (0.8, 1)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

In [30]:
# Get best results
print(f"Best Score: {mnb.best_score_} with parameters: {mnb.best_params_}")
optimized_mnb = mnb.best_estimator_

Best Score: 0.7823364879159459 with parameters: {'classify__alpha': 0.8, 'vectorize__max_df': 0.8, 'vectorize__ngram_range': (1, 2), 'vectorize__norm': 'l2', 'vectorize__use_idf': False}


In [31]:
# Evaluate model using test data
predictions = optimized_mnb.predict(X_test)
print(f"Accuracy score: {accuracy_score(y_test, predictions)}")
print(f"Confusion matrix: {confusion_matrix(y_test, predictions)}")
print(f"Classification report: \n{classification_report(y_test, predictions)}")

Accuracy score: 0.7839677419354839
Confusion matrix: [[ 86460  29649]
 [ 23927 107964]]
Classification report: 
              precision    recall  f1-score   support

           0       0.78      0.74      0.76    116109
           1       0.78      0.82      0.80    131891

   micro avg       0.78      0.78      0.78    248000
   macro avg       0.78      0.78      0.78    248000
weighted avg       0.78      0.78      0.78    248000



### SVG Classifier

In [16]:
# Create transformation pipeline
pipeline = Pipeline([
    ("vectorize", TfidfVectorizer()),
    ("classify", SVC()),
])

# Set parameters for Gridsearch
parameters = {"vectorize__use_idf": (True, False),
              "vectorize__ngram_range": [(1,1), (1,2)],
              "vectorize__max_df" : (1, .8),
              "vectorize__norm": ("l1", "l2"),
              "classify__kernel": ("linear", "rbf’")
             }
svc = GridSearchCV(pipeline, param_grid = parameters, n_jobs = -1, cv = 5, verbose = 1)

In [17]:
svc.fit(X_train,y_train)

Fitting 5 folds for each of 32 candidates, totalling 160 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


KeyboardInterrupt: 

In [ ]:
# Get best results
print(f"Best Score: {svc.best_score_} with parameters: {svc.best_params_}")
optimized_svc = svc.best_estimator_

In [ ]:
# Evaluate model using test data
predictions = optimized_svc.predict(X_test)
print(f"Accuracy score: {accuracy_score(y_test, predictions)}")
print(f"Confusion matrix: {confusion_matrix(y_test, predictions)}")
print(f"Classification report: \n{classification_report(y_test, predictions)}")

### Deep Learning Model

['twitter_model_mnb.joblib']

## Save Best Model

In [35]:
# Save best model
dump(optimized_mnb, 'twitter_model.joblib')

['twitter_model.joblib']